# QuantConnect API Integration

This notebook provides an interface to the QuantConnect API for financial data analysis.

In [ ]:
# Install required packages
%pip install requests pandas numpy matplotlib

In [ ]:
# Import libraries
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Setup plotting
%matplotlib inline
plt.style.use('ggplot')

print("Libraries imported successfully!")

In [ ]:
class QuantConnectAPI:
    """Simple QuantConnect API wrapper"""
    def __init__(self, api_key=None, api_secret=None):
        self.api_url = "https://api.quantconnect.com/api/v2"
        self.api_key = api_key
        self.api_secret = api_secret
        self.connected = False
        
        if api_key and api_secret:
            self.test_connection()
    
    def test_connection(self):
        """Test the API connection"""
        if not self.api_key or not self.api_secret:
            print("❌ API credentials not set. Use set_credentials(api_key, api_secret)")
            return False
            
        try:
            endpoint = "/projects/list"
            response = self._api_request("GET", endpoint)
            
            if response and "projects" in response:
                print(f"✅ Successfully connected to QuantConnect API")
                print(f"   Found {len(response['projects'])} projects")
                self.connected = True
                return True
            else:
                print("❌ Connection failed. Check credentials and try again.")
                return False
        except Exception as e:
            print(f"❌ API Error: {str(e)}")
            return False
    
    def set_credentials(self, api_key, api_secret):
        """Set API credentials"""
        self.api_key = api_key
        self.api_secret = api_secret
        return self.test_connection()
    
    def _api_request(self, method, endpoint, data=None):
        """Make an API request to QuantConnect"""
        headers = {
            'Content-Type': 'application/json'
        }
        
        url = f"{self.api_url}{endpoint}"
        
        # Add authentication parameters
        params = {
            "userId": self.api_key
        }
        
        if data is None:
            data = {}
        
        # Add API secret to the data
        data["token"] = self.api_secret
        
        try:
            if method.upper() == "GET":
                response = requests.get(url, headers=headers, params=params, json=data)
            elif method.upper() == "POST":
                response = requests.post(url, headers=headers, params=params, json=data)
            else:
                raise ValueError(f"Unsupported HTTP method: {method}")
            
            if response.status_code == 200:
                return response.json()
            else:
                print(f"❌ API Error: Status {response.status_code}")
                print(f"Response: {response.text}")
                return None
        except Exception as e:
            print(f"❌ Request Error: {str(e)}")
            return None
    
    def get_data(self, symbol, resolution="daily", start_date=None, end_date=None):
        """Get historical market data for a symbol"""
        if not self.connected:
            print("❌ Not connected. Call set_credentials() first.")
            return None
            
        # Set default dates if not provided
        if not end_date:
            end_date = datetime.now().strftime('%Y-%m-%d')
        if not start_date:
            # Default to 1 year ago
            start_date = (datetime.now() - timedelta(days=365)).strftime('%Y-%m-%d')
        
        # Prepare data request
        data = {
            "symbols": [symbol],
            "resolution": resolution,
            "start": start_date,
            "end": end_date
        }
        
        response = self._api_request("POST", "/data/read", data)
        
        if response and "data" in response and symbol in response["data"]:
            # Process the data into a DataFrame
            symbol_data = response["data"][symbol]
            
            if len(symbol_data) == 0:
                print(f"No data found for {symbol} in the specified date range")
                return pd.DataFrame()
            
            df = pd.DataFrame(symbol_data)
            df['time'] = pd.to_datetime(df['time'])
            df = df.set_index('time')
            
            print(f"✅ Retrieved {len(df)} records for {symbol}")
            return df
        else:
            print(f"❌ Failed to retrieve data for {symbol}")
            return None

# Create an instance of QuantConnectAPI
qc = QuantConnectAPI()

print("🚀 QuantConnect API utilities loaded successfully!")

## Connect to QuantConnect API

Replace with your actual API credentials:

In [ ]:
# Enter your API credentials here
API_KEY = "YOUR_API_KEY"
API_SECRET = "YOUR_API_SECRET"

# Connect to the API
qc.set_credentials(API_KEY, API_SECRET)

## Get Stock Data

Fetch historical stock data for analysis:

In [ ]:
# Get Apple stock data for the past year
apple_data = qc.get_data('AAPL')

# Display the first few rows
if apple_data is not None:
    apple_data.head()

## Visualize the Data

Create a chart of the stock price:

In [ ]:
# Plot Apple stock price
if apple_data is not None and len(apple_data) > 0:
    plt.figure(figsize=(12, 6))
    plt.plot(apple_data.index, apple_data['close'], label='AAPL Close')
    plt.title('Apple Stock Price')
    plt.xlabel('Date')
    plt.ylabel('Price ($)')
    plt.legend()
    plt.grid(True)
    plt.show()

## Advanced Analysis Example

Calculate moving averages and visualize:

In [ ]:
# Calculate moving averages
if apple_data is not None and len(apple_data) > 0:
    # Calculate 20-day and 50-day moving averages
    apple_data['MA20'] = apple_data['close'].rolling(window=20).mean()
    apple_data['MA50'] = apple_data['close'].rolling(window=50).mean()
    
    # Plot price with moving averages
    plt.figure(figsize=(14, 7))
    plt.plot(apple_data.index, apple_data['close'], label='AAPL Close', linewidth=1)
    plt.plot(apple_data.index, apple_data['MA20'], label='20-day MA', linewidth=1.5)
    plt.plot(apple_data.index, apple_data['MA50'], label='50-day MA', linewidth=1.5)
    plt.title('Apple Stock Price with Moving Averages')
    plt.xlabel('Date')
    plt.ylabel('Price ($)')
    plt.legend()
    plt.grid(True)
    plt.show()
    
    # Display summary statistics
    print("\nSummary Statistics:")
    print(apple_data[['close', 'MA20', 'MA50']].describe())